READ IMAGES FROM "SAMPLES" folder, in sequence.

Create a NumPy archieve (NPZ is a file format by numpy that provides storage of array data using gzip compression) named "sequence_array.npz", containing the sequence of the images from them 'Samples' folder, as a Numpy array.

(Requires packages: Pillow, Tensorflow=1.15, Keras)
(conda install Pillow tensorflow=1.15 keras)

In [ ]:
import numpy as np
from PIL import Image
import os
import time

IMAGE_PATH = './samples/'

WIDTH = 100
HEIGHT = 100
SEQUENCE = np.array([])
BASIC_SEQUENCE = np.array([])
NEXT_SEQUENCE = np.array([])
NUMBER = 0

def image_initialize(image):
    picture = Image.open(image)
    picture = picture.crop((243, 176, 1428, 1280))
    picture = picture.resize((WIDTH, HEIGHT), Image.ANTIALIAS)
    picture = picture.convert('L')
    picture.save('./temp/1.png')
    data = np.array(picture.getdata()).reshape(WIDTH, HEIGHT, 1)
    return data

for file in os.listdir(IMAGE_PATH):
    image_array = image_initialize(os.path.join(IMAGE_PATH, file))
    SEQUENCE = np.append(SEQUENCE, image_array)
    NUMBER += 1
    print(NUMBER)
    print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

SEQUENCE = SEQUENCE.reshape(NUMBER, WIDTH * HEIGHT)
for i in SEQUENCE:
    for j in range(int(len(i))):
        if i[j] < 50:
            i[j] = 0


np.savez('sequence_array.npz', sequence_array=SEQUENCE)
print('Data saved.')
print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

###################################
MODEL TO TRAIN THE CONV-LSTM NETWORK
###################################


In [ ]:
import numpy as np
from PIL import Image
import os
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
import time
from keras.utils import multi_gpu_model
from keras import optimizers

In [ ]:
WIDTH = 100
HEIGHT = 100
FRAMES = 16

SEQUENCE = np.load('sequence_array.npz')['sequence_array']  # load array
print(SEQUENCE[0])
print('Data loaded.')
print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

In [ ]:
NUMBER = len(SEQUENCE)

In [ ]:
##print (NUMBER)

In [ ]:
# step =1
SEQUENCE = SEQUENCE.reshape(NUMBER, WIDTH, HEIGHT, 1)
# step =2
SEQUENCE_2 = []
for i in range(int(NUMBER / 2)):
    SEQUENCE_2.append(SEQUENCE[2 * i])

# step = 3
SEQUENCE_3 = []
for i in range(int(NUMBER / 3)):
    SEQUENCE_3.append(SEQUENCE[3 * i])

In [26]:
for i in range(FRAMES):
    model = Sequential()

    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),input_shape=(None, WIDTH, HEIGHT, 1), padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=60, kernel_size=(3, 3), padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=60, kernel_size=(3, 3), padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3), padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(Conv3D(filters=1, kernel_size=(3, 3, 3), activation='sigmoid', padding='same', data_format='channels_last'))
    
    #sgd = optimizers.SGD(lr=0.01, clipvalue=0.5)
    #sgd = optimizers.SGD(lr=0.01, clipvalue=1)
    #seq.compile(loss='mean_squared_error', optimizer='adadelta')
    #model.compile(loss='binary_crossentropy', optimizer='adadelta')
    
    # For a multi-class classification problem
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
print("done")

### GIVES ERROR !!!! ###  WHY ?? ###
model.fit(BASIC_SEQUENCE[:10], NEXT_SEQUENCE[:10], batch_size=10, epochs=10, validation_split=0.05)
model.save('nice_model.h5')
print ("model saved")

done


ValueError: Error when checking input: expected conv_lst_m2d_397_input to have 5 dimensions, but got array with shape (0, 1)